In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import warnings
from cgi import test
from pathlib import Path
import time
from unittest import result
from pytorch_lightning import Trainer
from ray import tune
import torch
from train import LABEL_NAME
import datetime
from functools import partial
from vacances_scolaires_france import SchoolHolidayDates
from train import DataModule, Regressor

sn.set(rc={'figure.figsize':(11.7,8.27)})
%matplotlib inline
%load_ext autoreload
%autoreload 2

warnings.filterwarnings("ignore")

In [ ]:
df_list_station = pd.read_csv(os.path.join(data_folder,"liste_stations.csv"), sep=";", header=0)
df_list_station.head()

In [ ]:
df_prev_sans_obs2020 = pd.read_feather(os.path.join(data_folder, "df_prev_sans_obs2020.feather"))
df_prev_sans_obs2020



In [ ]:
df_grille_zoneclimat_fin18 = pd.read_feather(os.path.join(data_folder, "grille_zone_climatique_fin2018.feather"))
df_grille_zoneclimat_fin18.head(10)

In [ ]:
df_meteo_zone_eol = pd.read_feather(os.path.join(data_folder, "meteo_zone_echeance12_2016_2020_HRES_piEOL_smooth.feather"))
df_meteo_zone_eol.groupby('date_lancement_meteo').count()

In [ ]:
df_meteo_zone_eol = pd.read_feather(os.path.join(data_folder, "meteo_zone_echeance12_2016_2020_HRES_piEOL_smooth.feather"))
df_meteo_zone_eol

In [ ]:
df_meteo_zone_pv = pd.read_feather(os.path.join(data_folder, "meteo_zone_echeance12_2016_2020_HRES_piPV_smooth.feather"))
df_meteo_zone_pv

In [ ]:
df_prodpv_fc_q90 = pd.read_feather(os.path.join(data_folder, "productionPV_FC_cielclair_q90.feather"))
df_prodpv_fc_q90.head()

In [ ]:
# Proving that all input data have hourly resolution
px.line(df_conso_res_meteo[df_conso_res_meteo.echeance==1.0].sort_values('date_cible'),x='date_cible',y="prev")

## Training

In [ ]:
# Run train.py

## Output analysis

In [ ]:
pd.options.plotting.backend = "plotly"
import plotly.express as px
import plotly.graph_objects as go

from submit import prepare_submission


def plot_val_output(obs_type):
        # Plot val preds. NB: there are holes because of the split between train and val.
    exp_path = max(Path(f"./ray/{obs_type}/").glob('*experiment*'), key=os.path.getctime) # take latest
    results = tune.ExperimentAnalysis(experiment_checkpoint_path=exp_path,default_metric="val/loss",default_mode="min")
    print(f"""
            Preparing submission for {obs_type}...
            Using Experiment {exp_path}
            
            """)
    print(results.best_config)

    out = prepare_submission(obs_type, results, df_type='val')
    df_conso_sub = out[(out.quantile_niveau.isin([0.25, 0.75]))& (out.echeance==1.0)]
    df_conso_sub.sort_values(by='date_cible',inplace=True)

    fig = px.line(df_conso_sub, x='date_cible',y='prev_q',color='quantile_niveau', line_dash_sequence=['dash'])
    fig.add_trace(go.Scatter(x=df_conso_sub.date_cible,y=df_conso_sub.prev,name="prev",line=dict(width=4)))
    fig.add_trace(go.Scatter(x=df_conso_sub.date_cible,y=df_conso_sub.obs,name="obs",line=dict(width=4)))
    fig.update_traces(connectgaps=False) 
    return fig



In [ ]:
plot_val_output('consommation_residuelle')


In [ ]:
plot_val_output('consommation')

In [ ]:
plot_val_output('eolien')

In [ ]:
dm.df_val.info()

In [ ]:

def analyze_fiability(out):
    quantile_levels = np.arange(0.005,1.00,0.005)
    out['percentage'] = pd.NA
    # TODO: 1 select quantile
    # 2 compute % of distribution above it
    # 3 deduce loss
    # LOSS 
    out['percentage'] = out.apply(lambda row: (row.prev_q >= out.obs).sum()/len(out), axis=1)
    out['reliability'] = indicator(out['percentage'] - out['prev_q'])*10
    
    return out
analyze_fiability(out.head(1000))

In [ ]:
def show_features(obs_type):
    df = pd.read_hdf(f'features/{obs_type}.hdf')
    dm = DataModule(df, LABEL_NAME,10)
    dm.prepare_data()
    # Show feature columns
    print(dm.df_train.drop(columns=dm.cols_to_drop).columns)
show_features('photovoltaique')

## IDEAs
- coeff zone meteo ? altitude ? longitude ?
- train only on useful horizons ?
- add dvc 
- add given FC 

## TODO

- add fiabilité
- add meteo for conso


